In [1]:
import databento as db
import pandas as pd

In [2]:
API_KEY = "Your API Key"

client = db.Historical(API_KEY)

In [3]:
symbols = ["SR1N5", "SR1M5", "SR3H5", "SR3M5", "SR3U5", "SR3Z5", "SR3H6", "SR3M6", "SR3U6", "SR3Z6", "SR3H7", "SR3M7", "SR3U7", "SR3Z7", "SR3H8"] 
#["SR1.c.0", "SR1.c.1"] denotes continuous contracts  

In [12]:
stats = client.timeseries.get_range(
    dataset="GLBX.MDP3",
    schema="statistics",
    symbols=symbols,
    #stype_in="continuous",
    start="2025-03-19",
    end="2025-03-20"
).to_df()

In [13]:
definitions = client.timeseries.get_range(
    dataset="GLBX.MDP3",
    schema="definition",
    symbols=symbols,
    #stype_in="continuous",
    start="2025-03-19",
    end="2025-03-20"
).to_df()

In [14]:
stats = stats[
    stats.stat_type.isin([db.StatType.SETTLEMENT_PRICE])
]

In [15]:
stats["stat"] = stats["stat_type"].map(
    {
        db.StatType.SETTLEMENT_PRICE: "settlement price",
    },
)
stats["value"] = stats.apply(
    lambda r: r.price if r.stat_type == db.StatType.SETTLEMENT_PRICE else r.quantity,
    axis=1,
)

In [16]:
stats["ts_ref_date"] = stats["ts_ref"].dt.floor("D")

In [17]:
res = (
    stats.groupby(["ts_ref_date", "symbol", "stat_type"])
    # Select the last (most-recent) record for each unique `ts_ref` and `symbol`
    .last().reset_index()
    .pivot(columns="stat", index=["ts_ref_date", "symbol"], values="value")
)

In [18]:
combined_df = stats.merge(
    definitions,
    on="instrument_id",
    how="inner",
    suffixes=("", "_def"),
).set_index("ts_ref_date")

print(combined_df[["raw_symbol", "expiration", "price"]].head())

                          raw_symbol                expiration   price
ts_ref_date                                                           
2025-03-19 00:00:00+00:00      SR3Z5 2026-03-17 21:00:00+00:00  96.295
2025-03-19 00:00:00+00:00      SR3M7 2027-09-14 21:00:00+00:00  96.435
2025-03-19 00:00:00+00:00      SR3U6 2026-12-15 22:00:00+00:00  96.495
2025-03-19 00:00:00+00:00      SR3H8 2028-06-20 21:00:00+00:00  96.340
2025-03-19 00:00:00+00:00      SR3Z7 2028-03-14 21:00:00+00:00  96.375


In [19]:
print(res.xs("SR3H5", level="symbol"))

stat                       settlement price
ts_ref_date                                
2025-03-19 00:00:00+00:00           95.6875


In [20]:
final_data = combined_df[
    (combined_df['stat_flags'].astype(int) & 0x01) == 0 &
    (combined_df['stat_type'] == 7)
].copy()

final_data['date'] = final_data['ts_event'].dt.date

daily_settlements = final_data.sort_values('ts_event').groupby(['date', 'symbol']).last().reset_index()

pivot_price = daily_settlements.pivot(index='date', columns='symbol', values='price')
pivot_underlying = daily_settlements.pivot(index='date', columns='symbol', values='raw_symbol')

combined = pd.concat({'Raw Symbol': pivot_underlying, 'Settlement Price': pivot_price}, axis=1)

combined = combined.sort_index(axis=1, level=1)

combined

,Raw Symbol,Settlement Price,Raw Symbol,Settlement Price,Raw Symbol,Settlement Price,Raw Symbol,Settlement Price,Raw Symbol,Settlement Price,...,Raw Symbol,Settlement Price,Raw Symbol,Settlement Price,Raw Symbol,Settlement Price,Raw Symbol,Settlement Price,Raw Symbol,Settlement Price
symbol,SR1M5,SR1M5,SR1N5,SR1N5,SR3H5,SR3H5,SR3H6,SR3H6,SR3H7,SR3H7,...,SR3U6,SR3U6,SR3U7,SR3U7,SR3Z5,SR3Z5,SR3Z6,SR3Z6,SR3Z7,SR3Z7
date,,,,,,,,,,,,,,,,,,,,,
2025-03-19,SR1M5,95.795,SR1N5,95.875,SR3H5,95.6875,SR3H6,96.415,SR3H7,96.465,...,SR3U6,96.495,SR3U7,96.405,SR3Z5,96.295,SR3Z6,96.485,SR3Z7,96.375


In [21]:
output_file = "output_file_name.xlsx"

combined.to_excel(output_file, 
                  merge_cells=True,  
                  index=True)        

print(f" Exported successfully to {output_file}")

 Exported successfully to final_settlement_prices_0319.xlsx
